# Neural processes

In [7]:
import tensorflow as tf
from collections import namedtuple

In [10]:
def encoder_h(inputs: tf.Tensor, n_hidden_units: int, dim_r: int) -> tf.Tensor :
    """Map inputs (x_i, y_i) to r_i"""
    hidden_outputs = tf.layers.dense(inputs, n_hidden_units, activation=tf.nn.sigmoid, name='encoder_layer1', reuse=tf.AUTO_REUSE)
    r = tf.layers.dense(hidden_outputs, dim_repr, name="encoder_layer2", reuse=tf.AUTO_REUSE)
    return r

In [11]:
def aggregate_reprs(rs: tf.Tensor) -> tf.Tensor:
    """Aggregate the output of the encoder to a single representation"""
    mean = tf.reduce_mean(rs, axis=0)
    r = tf.reshape(mean, [1, -1])
    return r

In [17]:
GaussianParams = namedtuple('GaussianParams', ['mu', 'sigma'])

def get_z_params(input_r: tf.Tensor, dim_z: int) -> ZParams:
    mu = tf.layers.dense(input_r, dim_z, name="z_params_mu", reuse=tf.AUTO_REUSE)
    sigma = tf.layers.dense(input_r, dim_z, name="z_params_sigma", reuse=tf.AUTO_REUSE)
    sigma = tf.nn.softplus(sigma)
    return GaussianParams(mu, sigma)

In [18]:
def decoder_g(z_samples: tf.Tensor, x_star: tf.Tensor, n_hidden_units, noise_std: float=0.05) -> tf.Tensor:
    # inputs dimensions
    # z_sample has dim [n_draws, dim_z]
    # x_star has dim [N_star, dim_x]

    n_draws = z_samples.get_shape().as_list()[0]
    n_xs = x_star.get_shape().as_list()[0]
    
    # Repeat z samples for each x*
    z_samples_repeat = tf.expand_dims(z_samples, axis=1)
    z_samples_repeat = tf.tile(z_samples_repeat, [1, n_xs, 1])
    
    # Repeat x* for each z sample
    x_star_repeat = tf.expand_dims(x_star, axis=0)
    x_star_repeat = tf.tile(x_star_repeat, [n_draws, 1, 1])
    
    # Concatenate x* and z
    inputs = tf.concat([x_star_repeat, z_samples_repeat], axis=2)
    
    hidden = tf.layers.dense(inputs, n_hidden_units, activation=tf.nn.sigmoid, name="decoder_layer1", reuse=tf.AUTO_REUSE)
    
    # mu will be of the shape [N_star, n_draws]
    mu_star = tf.layers.dense(hidden, 1, name="decoder_layer2", reuse=tf.AUTO_REUSE)
    mu_star = tf.squeeze(mu_star, axis=2)
    mu_star = tf.transpose()
    
    sigma_star = tf.constant(noise_std, dtype=tf.float32)

    return GaussianParams(mu_star, sigma_star)

In [16]:
def xy_to_z_params(xs: tf.Tensor, ys: tf.Tensor, n_hidden_units_h: int, dim_r: int, dim_z: int):
    xys = tf.concat([xs, ys], axis=1)
    rs = encoder_h(xys, n_hidden_units_h, dim_r)
    r = aggregate_r(rs)
    z_params = get_z_params(r, dim_z)
    return z_params

In [19]:
def KLqp_gaussian(mu_q, sigma_q, mu_p, sigma_p):
    sigma2_q = tf.square(sigma_q) + 1e-16
    sigma2_p = tf.square(sigma_p) + 1e-16
    temp = sigma2_q / sigma2_p + tf.square(mu_q - mu_p) / sigma2_p - 1.0 + tf.log(sigma2_p / sigma2_q + 1e-16)
    return 0.5 * tf.reduce_sum(temp)

In [ ]:
def init_neural_process(x_context, y_context, x_target, y_target, dim_r, dim_z, n_hidden_units_h: int, n_hidden_units_g: int, learning_rate=0.001, n_draws=7):
    # Concatenate context and target
    x_all = tf.concat([x_context, x_target], axis=0)
    y_all = tf.concat([y_context, y_target], axis=0)
    
    # Map input to z
    z_context = xy_to_z_params(x_context, y_context, dim_r, dim_z)
    z_all = xy_to_z_params(x_all, y_all, n_hidden_units_h, dim_r, dim_z)
    
    # Sample z
    epsilon = tf.random_normal([n_draws, dim_z])
    # TODO: Should this use sigma from z_context?
    z_samples = tf.multiply(epsilon, z_all.sigma)
    
    # Map (z, x*) to y*
    y_pred_params = decoder_g(z_samples, x_target, n_hidden_units_g)